### Simple example

The `EPyT` python class, epanet, includes properties of the input network model, static properties, public functions, and local functions that directly call the `EPANET` shared object library or the executable. The full API with detailed examples is provided in the link. In the following paragraphs, we introduce a selection of the capabilities of the tool and an illustrative smart water research example.

Through `EPyT`, users can easily load EPANET network models, as demonstrated by importing the module and loading the `Net1.inp` network in a provided example.

### Initialize EPANET Python Toolkit (EPyT)

You should always begin with this command to import the toolkit.

[EPyT](https://github.com/OpenWaterAnalytics/EPyT) is available on [PyPI](https://pypi.org/project/epyt/) and can be installed via `pip install epyt`. To upgrade to the latest version if it's already installed, use `pip install --upgrade epyt`.

In [ ]:
%pip install epyt

In [ ]:
from epyt import epanet

In [ ]:
# Load EPANET Input File
G = epanet('Net1.inp')

In this example, `G` is an epanet object which can be defined mathematically as the set comprised of the network graph topology (such as nodes and links), structural parameters (such as pipe lengths, diameters, etc.), and functions (such as hydraulic solvers, etc.). `G` can also be shared between different functions as an argument.

When the object is constructed, the module reads the `EPANET` input network file and populates more than 500 object parameters. To view some of the parameters and receive assistance with respect to the methods, the user can use the following commands:

In [ ]:
# Lists all available functions and properties
dir(G)

In [ ]:
# Retrieve some examples for the function
help(G.getNodeElevations)

In [ ]:
# Retrieve Link diameters
diameters = G.getLinkDiameter()
print(diameters)

In [ ]:
# Retrieve Node elevations
elevations = G.getNodeElevations()
print(elevations)

In [ ]:
# Link diameter for links 2 & 10
diameters = G.getLinkDiameter([2, 10])
print(diameters)

In [ ]:
# Update the link 10 diameter from 100 to 90
G.setLinkDiameter(10, 90)
# Retrieve the diameter of link 10
n_diameter = G.getLinkDiameter(10)
print(n_diameter)

In [ ]:
# Solve hydraulics in library
# H = G.getComputedHydraulicTimeSeries()

# Solve quality dynamics in library
# Q = G.getComputedQualityTimeSeries()

# Solve all dynamics in library, create a binary file to store the computed values
R = G.getComputedTimeSeries()

In [ ]:
# Plot link flows and quality
hrs_time = R.Time / 3600
link_indices = [1, 3, 5, 10]
link_names = G.getLinkNameID(link_indices)
G.plot_ts(X=hrs_time, Y=R.Flow[:, link_indices],
          title=f'Flow, Link IDs: {link_names}', figure_size=[4, 3], legend_location='best',
          xlabel='Time (hrs)', ylabel=f'Flow ({G.units.LinkFlowUnits})',
          marker=None, labels=link_names, save_fig=True) #, filename='figures/paper_flows')
G.plot_ts(X=hrs_time, Y=R.LinkQuality[:, link_indices],
          title=f'Quality, Link IDs: {link_names}', legend_location='best',
          xlabel='Time (hrs)', ylabel=f'Quality', figure_size=[4, 3],
          marker=None, labels=link_names, save_fig=True) #, filename='figures/paper_link_quality')

# Plot node pressures and quality
node_indices = [2, 4, 6, 10]
node_names = G.getNodeNameID(node_indices)
G.plot_ts(X=hrs_time, Y=R.Pressure[:, node_indices], legend_location='best',
          title=f'Pressure, Node IDs: {node_names}', figure_size=[4, 3],
          xlabel='Time (hrs)', ylabel=f'Pressure ({G.units.NodePressureUnits})',
          marker=None, labels=node_names, save_fig=True) #, filename='figures/paper_pressures')

G.plot_ts(X=hrs_time, Y=R.NodeQuality[:, node_indices],
          title=f'Quality, Node IDs: {node_names}', legend_location='best',
          xlabel='Time (hrs)', ylabel=f'Quality', figure_size=[4, 3],
          marker=None, labels=node_names, save_fig=True) #, filename='figures/paper_node_quality')


#### Unload library

In [ ]:
G.unload()